In [ ]:
%load_ext autoreload
%autoreload 2

# Dask

> Dask Buffer

In [ ]:
#| default_exp storage.buffer.dask

In [ ]:
#| export
from __future__ import annotations
from configparser import ConfigParser
from dataclasses import dataclass
from typing import Optional, Tuple, Union
import logging
import numpy as np
import pandas as pd  # type: ignore

In [ ]:
#| export
from data_io_nbdev.config.drivers import Driver
from data_io_nbdev.config.vehicles import Truck
from data_io_nbdev.storage.pool.avro.avro import AvroPool
from data_io_nbdev.storage.pool.parquet import ParquetPool
from data_io_nbdev.data.core import (
    ObservationMeta,
    PoolQuery,
    veos_lifetime_end_date,
    veos_lifetime_start_date,
)
from data_io_nbdev.data.external.pandas_utils import (
    decode_episode_batch_to_padded_arrays,
)

2023-12-19 14:20:19.198547: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-19 14:20:19.251343: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-19 14:20:19.251380: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-19 14:20:19.251427: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-19 14:20:19.269807: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-19 14:20:19.290091: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [ ]:
#| export
from data_io_nbdev.storage.buffer.buffer import Buffer  # type: ignore

In [ ]:
#| export
@dataclass(kw_only=True)
class DaskBuffer(Buffer[pd.DataFrame]):
    """
    A Buffer connected with a data array file pool
    
    Args:
        recipe:  ConfigParser containing a folder for the data files and the ObservationMeta
        batch_size: the batch size for sampling
        driver: the driver
        truck: the subject of the experiment
        meta: the metadata of the overservation `ObservationMeta`
        torque_table_row_names: the names of the torque table rows, e.g. ['r0, r1, r2, r3, r4, r5, r6, r7, r8, r9']
        pool: the pool to sample from, default is `ParquetPool`
        query: the query to sample from the pool, default is `PoolQuery`
        logger: the logger
        dict_logger: the dictionary logger
    """

    recipe: ConfigParser  # field(default_factory=get_filemeta_config)
    batch_size: int  # 0
    driver: Driver  # field(default_factory=Driver)
    truck: Truck  # field(default_factory=Truck)
    meta: ObservationMeta  # field(default_factory=ObservationMeta)
    torque_table_row_names: list[str]  # field(default_factory=list)
    pool: Optional[
        Union[ParquetPool, AvroPool]
    ] = None  # field(default_factory=ParquetPool)  # cannot initialize an ABC of DaskPool
    query: Optional[PoolQuery] = None  # field(default_factory=PoolQuery)
    logger: Optional[logging.Logger] = None
    dict_logger: Optional[dict] = None

    def __post_init__(self):
        """set logger and load the pool to the buffer """
        self.logger = self.logger.getChild("main").getChild("arrow buffer")
        self.logger.propagate = True
        if not self.torque_table_row_names:
            self.torque_table_row_names = self.meta.get_torque_table_row_names()
        super().__post_init__()
        self.load()

    def load(self):
        """load the pool to the buffer"""
        # check if datafolder exists and have valid ini file for recipe
        # if not, create a new one
        # if yes, load the recipe and compare with the realtime truck signal specs
        # if not matching, raise error
        # if matching, continue

        # DaskPool differs from MongoPool in that it stores RECORD in parquet files and EPISODE in avro files,
        # and it uses different query classes for each
        if self.recipe["DEFAULT"]["coll_type"] == "RECORD":
            self.query = PoolQuery(
                vehicle=self.truck.vid,
                driver=self.driver.pid,
                episodestart_start=veos_lifetime_start_date.to_pydatetime(),
                episodestart_end=veos_lifetime_end_date.to_pydatetime(),
                timestamp_start=veos_lifetime_start_date.to_pydatetime(),
                timestamp_end=veos_lifetime_end_date.to_pydatetime(),
            )
            self.pool = ParquetPool(
                recipe=self.recipe,
                query=self.query,
                meta=self.meta,
                logger=self.logger,
                dict_logger=self.dict_logger,
            )
        else:  # coll_type == "EPISODE"
            self.query = PoolQuery(
                vehicle=self.truck.vid,
                driver=self.driver.pid,
                episodestart_start=veos_lifetime_start_date.to_pydatetime(),
                episodestart_end=veos_lifetime_end_date.to_pydatetime(),
                seq_len_from=0,
                seq_len_to=int(1e9),
            )
            self.pool = AvroPool(
                recipe=self.recipe,
                query=self.query,
                meta=self.meta,
                logger=self.logger,
                dict_logger=self.dict_logger,
            )

        if self.pool.cnt != 0:
            batch_1 = self.pool.sample(size=1, query=self.query)

        number_states, number_actions = self.meta.get_number_of_states_actions()
        self.logger.info(
            f'Connected to DaskPool {self.recipe["DEFAULT"]["data_folder"]}, '
            f"record number {self.pool.cnt}, "
            f"num_states: {number_states}, num_actions: {number_actions}",
            extra=self.dict_logger,
        )

    def decode_batch_records(
        self, batch: pd.DataFrame
    ) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
        """
        Decode the batch records from dask DataFrame to numpy arrays
        
        sample from parquet pool through dask give dask DataFrame, no heavy decoding required just slicing and converting to numpy array
        
        Arg:
        
            batch: the batch of records from dask DataFrame
            
        Return:
            
                states: the states of the batch
                actions: the actions of the batch
                rewards: the rewards of the batch
                nstates: the next states of the batch
        """

        idx = pd.IndexSlice
        npa_states = batch.loc[
            :, idx["state", ["velocity", "thrust", "brake"]]  # type: ignore
        ].values.astype(
            np.float32
        )  # same order as inference!! transpose values not necessary!
        # as each row of batch already corresponds to a tuple with the timestamp

        npa_actions = batch.loc[
            :, idx["action", self.torque_table_row_names]  # type: ignore
        ].values.astype(np.float32)

        npa_rewards = batch.loc[:, idx["reward", "work"]].values.astype(np.float32)  # type: ignore

        npa_nstates = batch.loc[
            :, idx["nstate", ["velocity", "thrust", "brake"]]  # type: ignore
        ].values.astype(
            np.float32
        )  # same order as inference!! transpose values not necessary!
        # as each row of batch already corresponds to a tuple with the timestamp

        return npa_states, npa_actions, npa_rewards, npa_nstates

    def sample(self) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
        """
        Sampling from the MongoDB pool
        """

        batch = self.pool.sample(size=self.batch_size, query=self.query)

        if self.recipe["DEFAULT"]["coll_type"] == "RECORD":
            states, actions, rewards, nstates = self.decode_batch_records(batch)
        else:  # coll_type == "EPISODE"
            (
                states,
                actions,
                rewards,
                nstates,
            ) = decode_episode_batch_to_padded_arrays(
                batch, self.torque_table_row_names
            )

        return states, actions, rewards, nstates

    def close(self):
        self.pool.close()

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
show_doc(DaskBuffer.__post_init__)

---

[source](https://github.com/Binjian/data-io-nbdev/tree/main/blob/main/data_io_nbdev/storage/buffer/dask.py#L65){target="_blank" style="float:right; font-size:smaller"}

### DaskBuffer.__post_init__

>      DaskBuffer.__post_init__ ()

set logger and load the pool to the buffer

In [ ]:
show_doc(DaskBuffer.load)

---

[source](https://github.com/Binjian/data-io-nbdev/tree/main/blob/main/data_io_nbdev/storage/buffer/dask.py#L74){target="_blank" style="float:right; font-size:smaller"}

### DaskBuffer.load

>      DaskBuffer.load ()

load the pool to the buffer

In [ ]:
show_doc(DaskBuffer.sample)

---

[source](https://github.com/Binjian/data-io-nbdev/tree/main/blob/main/data_io_nbdev/storage/buffer/dask.py#L171){target="_blank" style="float:right; font-size:smaller"}

### DaskBuffer.sample

>      DaskBuffer.sample ()

Sampling from the MongoDB pool

In [ ]:
show_doc(DaskBuffer.close)

---

[source](https://github.com/Binjian/data-io-nbdev/tree/main/blob/main/data_io_nbdev/storage/buffer/dask.py#L192){target="_blank" style="float:right; font-size:smaller"}

### DaskBuffer.close

>      DaskBuffer.close ()

close the pool, for destructor

In [ ]:
show_doc(DaskBuffer.decode_batch_records)

---

[source](https://github.com/Binjian/data-io-nbdev/tree/main/blob/main/data_io_nbdev/storage/buffer/dask.py#L128){target="_blank" style="float:right; font-size:smaller"}

### DaskBuffer.decode_batch_records

>      DaskBuffer.decode_batch_records (batch:pandas.core.frame.DataFrame)

Decode the batch records from dask DataFrame to numpy arrays

sample from parquet pool through dask give dask DataFrame, no heavy decoding required just slicing and converting to numpy array

Arg:

    batch: the batch of records from dask DataFrame

Return:

        states: the states of the batch
        actions: the actions of the batch
        rewards: the rewards of the batch
        nstates: the next states of the batch

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()